In [7]:
import config
import pandas as pd
import numpy as np
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import pickle
from matplotlib import pyplot
from IPython.display import IFrame
import traceback
import logging


In [8]:
##orginal song database.
song_database = pd.read_csv('song_database.csv')
##Billboard top 100.
hot_list = pd.read_csv('top100_.csv')
##orginal song database with clusters.
database_cluster = pd.read_csv('database_cluster.csv')

In [9]:
song = input('Hello, can you share me a song you are listening? ')

Hello, can you share me a song you are listening? 


In [10]:
song = song.lower()
if song in hot_list.song.values:
    recommand_100 = hot_list.sample()
    song_100 = ''.join(recommand_100.song.values)
    artist_100 = ''.join(recommand_100.artist.values)
    print('Cool! Your song is in the top 100. Let me recommand another song from top 100. The song is', song_100.capitalize(), 'by', artist_100.capitalize())
    index_100 = recommand_100.index
    track_id = song_database.loc[index_100, 'id'].values[0]

    
else:
    try:
        sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id, client_secret= config.client_secret))
        selected_song = sp.search(q=song, limit=1,market="GB") 
        selected_song_feature = sp.audio_features(selected_song["tracks"]["items"][0]["uri"])[0]
        new_songlist = pd.read_csv('new_songlist.csv')
    
        def load(filename = "filename.pickle"): 
            try: 
                with open(filename, "rb") as f: 
                    return pickle.load(f) 
            except FileNotFoundError: 
                print("File not found!") 
                
    except Exception:
        recommand_random = hot_list.sample()
        song_random = ''.join(recommand_random.song.values)
        artist_random = ''.join(recommand_random.artist.values)
        print('Your song can not be found in our database, let me recommand you a hit song', song_random.capitalize(), 'by', artist_random.capitalize())
        index_random = recommand_random.index
        track_id = song_database.loc[index_random, 'id'].values[0]      
    
    
    


HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': '', 'limit': 1, 'offset': 0, 'type': 'track', 'market': 'GB'} returned 400 due to No search query


Your song can not be found in our database, let me recommand you a hit song All i want for christmas is you by Mariah carey


In [11]:
selected_song_feature

{'danceability': 0.696,
 'energy': 0.921,
 'key': 0,
 'loudness': -3.755,
 'mode': 1,
 'speechiness': 0.0363,
 'acousticness': 0.00314,
 'instrumentalness': 5.24e-05,
 'liveness': 0.0842,
 'valence': 0.714,
 'tempo': 119.001,
 'type': 'audio_features',
 'id': '0SiywuOBRcynK0uKGWdCnn',
 'uri': 'spotify:track:0SiywuOBRcynK0uKGWdCnn',
 'track_href': 'https://api.spotify.com/v1/tracks/0SiywuOBRcynK0uKGWdCnn',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0SiywuOBRcynK0uKGWdCnn',
 'duration_ms': 294573,
 'time_signature': 4}

In [12]:
model = load(filename = "Model/kmeans_7.pickle")

scaler2 = load("Model/scaler.pickle")
scaler2

StandardScaler()

In [13]:
model

KMeans(n_clusters=7, random_state=1234)

In [14]:
column_names = ['acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature','valence']
empty_database = pd.DataFrame(np.random.randint(0,1,size=(1, 13)), columns = column_names)


In [15]:
for a in column_names:
    empty_database[a] = selected_song_feature[a]


In [16]:
empty_database_scaled = scaler2.transform(empty_database)
empty_database_scaled_df = pd.DataFrame(empty_database_scaled, columns = empty_database.columns)
empty_database_scaled_df

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,-1.071635,0.675552,0.663991,1.435786,-0.51751,-1.466243,-0.647589,1.039007,0.750669,-0.484054,-0.032066,0.219065,1.043255


In [17]:
song_cluster = model.predict(empty_database_scaled_df)

In [18]:
song_cluster

array([2], dtype=int32)

In [19]:
recommand_song = database_cluster[database_cluster['cluster'] == song_cluster[0]].sample()

In [20]:
recommand_song

,Unnamed: 0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,cluster
22159,22159,0.14,0.636,175465.0,0.875,0.0,6.0,0.355,-5.151,0.0,0.0391,149.939,4.0,0.846,2


In [21]:
a = recommand_song.index

In [22]:
track_id = song_database.loc[a, 'id'].values[0]


In [23]:
from IPython.display import IFrame


IFrame(src=f"https://open.spotify.com/embed/track/{track_id}",
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

In [25]:
labels = model.predict(empty_database_scaled_df)

labels

array([2], dtype=int32)